In [71]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from dateutil import parser

In [72]:
date='2023-11-01'
format=datetime.strptime(date, "%Y-%m-%d")
# print(format)
formatted_date_string = format.strftime("%Y/%m/%d")
print(formatted_date_string)

2023/11/01


In [73]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://republika.co.id/index/50/2023/11/01"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
articles = soup.find_all('li', {"class": "list-group-item list-border conten1"})
for article in articles:
    link = article.find('a')['href']
    print(link)

https://ekonomi.republika.co.id/berita/s3g97c502/mendag-sebut-larangan-transaksi-di-tiktok-tindakan-adil
https://internasional.republika.co.id/berita/s3g92v383/who-ribuan-orang-di-gaza-menderita-penyakit-kronis
https://tv.republika.co.id/berita/s3g90n418/gus-yahya-apresiasi-jokowi-pertemukan-ketiga-capres
https://rejogja.republika.co.id/berita/s3fxza291/inflasi-kota-malang-pada-oktober-2023-alami-peningkatan
https://khazanah.republika.co.id/berita/s3g5kv451/turki-kenakan-biaya-masuk-ke-masjid-hagia-sophia-bagi-pengunjung-asing
https://ameera.republika.co.id/berita/s3g84e463/perubahan-perilaku-pada-remaja-akibat-stres-jadi-rambu-bagi-orang-tua
https://visual.republika.co.id/berita/s3gelt375/bmw-indonesia-pamerkan-deretan-kendaraan-listrik-unggulannya
https://internasional.republika.co.id/berita/s3g8tv383/badan-intelijen-korsel-peluncuran-satelit-ketiga-korut-kemungkinan-tak-akan-gagal-lagi
https://rejabar.republika.co.id/berita/s3g8tl396/polisi-periksa-tiga-montir-bengkel-viral-di-cipan

In [74]:
def scrape_links(date,page_number):
    format=datetime.strptime(date, "%Y-%m-%d")
# print(format)
    formatted_date_string = format.strftime("%Y/%m/%d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://republika.co.id/index/{page_number}/{formatted_date_string}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('li', {"class": "list-group-item list-border conten1"})
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
    print(f"Scraped {len(links)} links from page {page_number}")
    
    return links

In [75]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 0
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 50

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [76]:
scrape_link_per_day('2023-11-01')

Scraped 50 links from page 0
Scraped 50 links from page 50
Scraped 50 links from page 100
Scraped 50 links from page 150
Scraped 50 links from page 200
Scraped 50 links from page 250
Scraped 50 links from page 300
Scraped 50 links from page 350
Scraped 50 links from page 400
Scraped 50 links from page 450
Scraped 50 links from page 500
Scraped 50 links from page 550
Scraped 50 links from page 600
Scraped 50 links from page 650
Scraped 0 links from page 700


['https://news.republika.co.id/berita/s3fey2349/yuk-penuhi-stadion-si-jalak-harupat-tiket-piala-dunia-u17-paling-murah-hanya-rp-30-ribu',
 'https://rejogja.republika.co.id/berita/s3fewp399/seluruh-kecamatan-di-yogyakarta-disebut-miliki-prevalensi-stunting-rendah',
 'https://sport.republika.co.id/berita/s3fetw438/pengamat-tak-mudah-bagi-indonesia-capai-target-16-besar-piala-dunia-u17',
 'https://rejabar.republika.co.id/berita/s3fep1432/kabel-alat-pemantau-aktivitas-gunung-tangkuban-parahu-hilang',
 'https://internasional.republika.co.id/berita/s3feli335/biden-terancam-kehilangan-suara-komunitas-muslim-di-pilpres-2024',
 'https://rejogja.republika.co.id/berita/s3feif399/ada-potensi-turun-hujan-di-sebagian-wilayah-jatim-ini-perkiraan-cuaca-bmkg',
 'https://khazanah.republika.co.id/berita/s3feha349/baznas-terima-bantuan-untuk-palestina-dari-laz-cahaya-quran-rabbani',
 'https://tv.republika.co.id/berita/s3fed3467/demo-bela-palestina-meluas-ke-kampuskampus-di-as',
 'https://visual.republika.

In [87]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('div', {"class": "max-card__title"})
                    if title_elem:
                        title_text = title_elem.find('h1')
                        title_text=title_text.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "date date-item__headline"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                        # Map Indonesian day names to English day names

                        # Define a regular expression pattern to extract date components
                        pattern = re.compile(r'(\d{2})\s+(\w{3})\s+(\d{4})\s+(\d{2}:\d{2})\s+WIB')

                        # Use regex to extract components
                        match = pattern.search(date_text)

                        if match:
                            day, month, year, time = match.groups()

                            # Map month abbreviations to month numbers
                            month_mapping = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
                                            'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

                            # Format the date string
                            formatted_date_string = f'{year}-{month_mapping[month]}-{day}'

                            # Parse the formatted date string into a datetime object
                           

                        else:
                            print("Date string does not match expected format.")
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "article-content"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date_string,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [88]:
url='https://news.republika.co.id/berita/s4tm48484/kontak-tembak-dengan-kst-di-nduga-empat-prajurit-tni-gugur'
data_repulika = scrape_url(url)

In [89]:
print(data_repulika)

{'title': 'Kontak Tembak dengan KST di Nduga, Empat Prajurit TNI Gugur', 'date': '2023-11-28', 'content': 'REPUBLIKA.CO.ID, NDUGA -- Empat personel Tentara Nasional Indonesia (TNI) gugur dalam kontak tembak dengan kelompok separatis teroris (KST) di Kabupaten Nduga, Provinsi Papua Pegunungan. Panglima TNI Jenderal Agus Subiyanto menyampaikan, Praka YL, Praka DB, Pratu MF, dan Prada DA meninggal dunia saat baku tembak dalam operasi di Distrik Paro. Tiga personel lainnya, yakni Serda AH, Pratu MI, dan Praka BS mengalami luka tembak. Jenderal Agus tak menjelaskan, kapan peristiwa kontak tembak itu terjadi. Menurut dia, keempat jenazah personel TNI yang gugur itu sudah dievakuasi.Mereka juga sudah dipulangkan ke keluarga masing-masing di Jawa Barat (Jabar) dan di Jawa Tengah (Jateng) untuk dimakamkan. "Bahwa keempat prajurit yang gugur tersebut, akan mendapatkan kenaikan pangkat luar biasa (KPLB) atas jasa-jasanya dalam pelaksanaan tugasnya di Papua," kata Agus di Jakarta, Selasa (28/11/20